# **AlexNet**

## General Imports

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Choosing the device

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on GPU(cuda)")
else:
    device = torch.device("cpu")
    print("Running on CPU")

## The AlexNet architecture

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4)
        self.conv2 = nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,stride=1,padding=2)
        self.conv3 = nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1)
        self.conv4 = nn.Conv2d(in_channels=384,out_channels=384,kernel_size=3,stride=1,padding=1)
        self.conv5 = nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,stride=1,padding=1)

        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2)
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2)
        self.pool3 = nn.MaxPool2d(kernel_size=3,stride=2)

        self.fc1 = nn.Linear(in_features=9216,out_features=4096)
        self.fc2 = nn.Linear(in_features=4096,out_features=4096)
        self.fc3 = nn.Linear(in_features=4096,out_features=1000)


    def forward(self,x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x= self.pool1(x)
        x = nn.ReLU()(x)

        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool2(x)
        x = nn.ReLU()(x)

        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.conv4(x)
        x = nn.ReLU()(x)
        x = self.conv5(x)
        x = nn.ReLU()(x)
        x = self.pool3(x)
        x = nn.ReLU()(x)

        x = nn.Flatten()(x)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.fc3(x)

        output = torch.nn.functional.softmax(x,dim=1)
        return output

## Training and Testing functions

In [ ]:
def train(model, device, train_loader, optimizer, epoch, print_step):
    for batch_idx, (img, label) in enumerate(train_loader):
        img, label = img.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(img)
        
        loss = torch.nn.CrossEntropyLoss()(output, label)
        loss.backward()

        optimizer.step()
        if batch_idx % print_step == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(img), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for img, label in test_loader:
            img, label = img.to(device), label.to(device)
            output = model(img)
            test_loss += torch.nn.CrossEntropyLoss(reduction='sum')(output, label).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Hyperparameters

In [ ]:
lr = 1e-2
momentum = 0.9
gamma = 0.9
epochs = 10
batch_size=64

## Loading the data

In [ ]:
mnist_train = datasets.MNIST(root='../data',train=True,download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root='../data',train=False,transform=transforms.ToTensor())

cifar100_train = datasets.CIFAR100(root='../data',train=True,download=True,transform=transforms.ToTensor())
cifar100_test = datasets.MNIST(root='../data',train=False,transform=transforms.ToTensor())

imagenet_train = datasets.ImageNet(root='../data',train=True,download=True, transform=transforms.ToTensor())
imagenet_test = datasets.ImageNet(root='../data',train=False,transform=transforms.ToTensor())

## Training the model

In [ ]:
model = AlexNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

train_loader = DataLoader(imagenet_train, batch_size=batch_size)
test_loader = DataLoader(imagenet_test, batch_size=batch_size)

for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch, 128)
        test(model, device, test_loader)
        scheduler.step()

torch.save(model.state_dict(), "AlexNet.pt")